In [ ]:
import math
import os
import shutil
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import pydicom
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
random_stat = 5675
np.random.seed(random_stat)

## 1. Clone and Build YOLOv3

In [ ]:
!git clone https://github.com/pjreddie/darknet.git

# Build gpu version darknet
!cd darknet && sed '1 s/^.*$/GPU=1/; 2 s/^.*$/CUDNN=1/' -i Makefile

# -j <The # of cpu cores to use>. Chang 999 to fit your environment. Actually i used '-j 50'.
!cd darknet && make -j 999 -s
!cp darknet/darknet darknet_gpu

## 2. Data Preparation for YOLOv3


In [ ]:
DATA_DIR = "../input"

train_dcm_dir = os.path.join(DATA_DIR, "stage_2_train_images")
test_dcm_dir = os.path.join(DATA_DIR, "stage_2_test_images")

img_dir = os.path.join(os.getcwd(), "images")  # .jpg
label_dir = os.path.join(os.getcwd(), "labels")  # .txt
metadata_dir = os.path.join(os.getcwd(), "metadata") # .txt

# YOLOv3 config file directory
cfg_dir = os.path.join(os.getcwd(), "cfg")
# YOLOv3 training checkpoints will be saved here
backup_dir = os.path.join(os.getcwd(), "backup")

for directory in [img_dir, label_dir, metadata_dir, cfg_dir, backup_dir]:
    if os.path.isdir(directory):
        continue
    os.mkdir(directory)

In [ ]:
annots = pd.read_csv(os.path.join(DATA_DIR, "stage_2_train_labels.csv"))
annots.head()

### 2.2. Generate images and labels for training YOLOv3


In [ ]:
def save_img_from_dcm(dcm_dir, img_dir, patient_id):
    img_fp = os.path.join(img_dir, "{}.jpg".format(patient_id))
    if os.path.exists(img_fp):
        return
    dcm_fp = os.path.join(dcm_dir, "{}.dcm".format(patient_id))
    img_1ch = pydicom.read_file(dcm_fp).pixel_array
    img_3ch = np.stack([img_1ch]*3, -1)

    img_fp = os.path.join(img_dir, "{}.jpg".format(patient_id))
    cv2.imwrite(img_fp, img_3ch)
    
def save_label_from_dcm(label_dir, patient_id, row=None):
    # rsna defualt image size
    img_size = 1024
    label_fp = os.path.join(label_dir, "{}.txt".format(patient_id))
    
    f = open(label_fp, "a")
    if row is None:
        f.close()
        return

    top_left_x = row[1]
    top_left_y = row[2]
    w = row[3]
    h = row[4]
    
    # 'r' means relative. 'c' means center.
    rx = top_left_x/img_size
    ry = top_left_y/img_size
    rw = w/img_size
    rh = h/img_size
    rcx = rx+rw/2
    rcy = ry+rh/2
    
    line = "{} {} {} {} {}\n".format(0, rcx, rcy, rw, rh)
    
    f.write(line)
    f.close()
        
def save_yolov3_data_from_rsna(dcm_dir, img_dir, label_dir, annots):
    for row in tqdm(annots.values):
        patient_id = row[0]

        img_fp = os.path.join(img_dir, "{}.jpg".format(patient_id))
        if os.path.exists(img_fp):
            save_label_from_dcm(label_dir, patient_id, row)
            continue

        target = row[5]
        # Since kaggle kernel have samll volume (5GB ?), I didn't contain files with no bbox here.
        if target == 0:
            continue
        save_label_from_dcm(label_dir, patient_id, row)
        save_img_from_dcm(dcm_dir, img_dir, patient_id)

In [ ]:
save_yolov3_data_from_rsna(train_dcm_dir, img_dir, label_dir, annots)

###  Generate train/val file path list (.txt)
* We should give the list of image paths to YOLO. two seperate list textfiles for training images and validation images.

In [ ]:
def write_train_list(metadata_dir, img_dir, name, series):
    list_fp = os.path.join(metadata_dir, name)
    with open(list_fp, "w") as f:
        for patient_id in series:
            line = "{}\n".format(os.path.join(img_dir, "{}.jpg".format(patient_id)))
            f.write(line)

In [ ]:
# Following lines do not contain data with no bbox
patient_id_series = annots[annots.Target == 1].patientId.drop_duplicates()

tr_series, val_series = train_test_split(patient_id_series, test_size=0.1, random_state=random_stat)
print("The # of train set: {}, The # of validation set: {}".format(tr_series.shape[0], val_series.shape[0]))

# train image path list
write_train_list(metadata_dir, img_dir, "tr_list.txt", tr_series)
# validation image path list
write_train_list(metadata_dir, img_dir, "val_list.txt", val_series)

### Create test image and labels for YOLOv3

In [ ]:
def save_yolov3_test_data(test_dcm_dir, img_dir, metadata_dir, name, series):
    list_fp = os.path.join(metadata_dir, name)
    with open(list_fp, "w") as f:
        for patient_id in series:
            save_img_from_dcm(test_dcm_dir, img_dir, patient_id)
            line = "{}\n".format(os.path.join(img_dir, "{}.jpg".format(patient_id)))
            f.write(line)

In [ ]:
test_dcm_fps = list(set(glob.glob(os.path.join(test_dcm_dir, '*.dcm'))))
test_dcm_fps = pd.Series(test_dcm_fps).apply(lambda dcm_fp: dcm_fp.strip().split("/")[-1].replace(".dcm",""))

save_yolov3_test_data(test_dcm_dir, img_dir, metadata_dir, "te_list.txt", test_dcm_fps)

### Plot a sample test Image

In [ ]:
ex_patient_id = test_dcm_fps[0]
ex_img_path = os.path.join(img_dir, "{}.jpg".format(ex_patient_id))

plt.imshow(cv2.imread(ex_img_path))

### Download Pre-trained Model
For training, we would download the pre-trained model weights(darknet53.conv.74) using following wget command. 

In [ ]:
!wget -q https://pjreddie.com/media/files/darknet53.conv.74

* ## 4. Training YOLOv3

In [ ]:
!./darknet_gpu detector train cfg/rsna.data cfg/rsna_yolov3.cfg_train darknet53.conv.74 -i 0 | tee train_log.txt

 Evaluating the performance of the model
We are evaluating the models based on the below metrics

Accuracy
Precision
Recall
F1 score
Mean IoU 

In [ ]:
# Make predictions on test images, write out sample submission
def predict_val(image_fps, filepath='val_sample.csv', min_conf=0.95):
    # assume square image
    resize_factor = ORIG_SIZE / config.IMAGE_SHAPE[0]
    #resize_factor = ORIG_SIZE
    with open(filepath, 'w') as file:
        file.write("patientId,score,x1,y1,width,height\n")

        for image_id in tqdm(image_fps):
            ds = pydicom.read_file(image_id)
            image = ds.pixel_array
            # If grayscale. Convert to RGB for consistency.
            if len(image.shape) != 3 or image.shape[2] != 3:
                image = np.stack((image,) * 3, -1)
            image, window, scale, padding, crop = utils.resize_image(
                image,
                min_dim=config.IMAGE_MIN_DIM,
                min_scale=config.IMAGE_MIN_SCALE,
                max_dim=config.IMAGE_MAX_DIM,
                mode=config.IMAGE_RESIZE_MODE)

            patient_id = os.path.splitext(os.path.basename(image_id))[0]

            results = model.detect([image])
            r = results[0]

            assert( len(r['rois']) == len(r['class_ids']) == len(r['scores']) )
            if len(r['rois']) != 0:
              num_instances = len(r['rois'])
              score = []  
              for i in range(num_instances):
                    score.append(r['scores'][i])
                    
                    if r['scores'][i] >= min_conf:
                      out_str = ""
                      out_str += patient_id
                      out_str += ","
                      out_str += ""
                      out_str += str(round(r['scores'][i], 2))
                      out_str += ","
                      x1 = r['rois'][i][1]
                      out_str += ""
                      out_str += str(x1*resize_factor)
                      out_str += ","
                      y1 = r['rois'][i][0]
                      out_str += ""
                      out_str += str(y1*resize_factor)
                      out_str += ","
                      width = r['rois'][i][3] - x1
                      out_str += ""
                      out_str += str(width*resize_factor)
                      out_str += ","
                      height = r['rois'][i][2] - y1
                      out_str += ""
                      out_str += str(height*resize_factor)
                      out_str += ","
                      file.write(out_str+"\n")
                    
              if max(score) < min_conf:
                      out_str = ""
                      out_str += patient_id
                      out_str += ","
                      file.write(out_str+"\n")  
            
            else:
                out_str = ""
                out_str += patient_id
                out_str += ","
                file.write(out_str+"\n")

In [ ]:
val_sample_fp = os.path.join(ROOT_DIR, 'val_sample.csv')
predict_val(image_fps_val, filepath=val_sample_fp)
print(val_sample_fp)

In [ ]:
col_names = ["patientId","score_pred","x_pred","y_pred","width_pred","height_pred"]
val_pred = pd.read_csv(val_sample_fp, delimiter = ",", names = col_names)
val_pred.drop(val_pred.index[0], inplace = True)
val_pred.sort_values(by = "patientId", inplace = True)
val_pred

In [ ]:
validation_list = set(val_pred['patientId'])
val_actuals = anns[anns['patientId'].isin(validation_list)]
val_actuals.sort_values(by = "patientId", inplace = True)
len(validation_list)

In [ ]:
val_compare = pd.merge(val_pred, val_actuals, how='inner', on= 'patientId', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [ ]:
val_compare.fillna(0, inplace = True)

In [ ]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 

In [ ]:
val_compare['score_pred'] = val_compare['score_pred'].astype(float)
val_compare['x_pred'] = val_compare['x_pred'].astype(float)
val_compare['y_pred'] = val_compare['y_pred'].astype(float)
val_compare['height_pred'] = val_compare['height_pred'].astype(float)
val_compare['width_pred'] = val_compare['width_pred'].astype(float)

In [ ]:
results = confusion_matrix(val_compare['Target'], val_compare['score_pred'].round())  

In [ ]:
# Defining function to find IoU scores
def IOU(x,y,width,height,x_pred,y_pred,width_pred,height_pred):
    w_intersection = min(x + width, x_pred + width_pred) - max(x, x_pred)
    h_intersection = min(y + height, y_pred + height_pred) - max(y, y_pred)
    if w_intersection <= 0 or h_intersection <= 0: # No overlap
        return 0
    I = w_intersection * h_intersection
    U = width * height + width_pred * height_pred - I # Union = Total Area - I
    return I / U

In [ ]:
val_compare['IoU'] = val_compare.apply(lambda x: IOU(x['x'],x['y'],x['width'],x['height'],x['x_pred'],x['y_pred'],x['width_pred'],x['height_pred']),axis=1)

In [ ]:
print("Evaluation Metrics")
print(classification_report(val_compare['Target'], val_compare['score_pred'].round()))
print("confusion matrix")
print(results)
print("IoU Scores")
val_compare.groupby('Target')["IoU"].mean()